In [14]:
import pandas as pd

data_path = "/Users/sarahSTACS/DSA4264-Detoxify/kedro-data/data/03_primary/concatenated_texts_data.csv/2024-10-08T12.08.08.170Z/concatenated_texts_data.csv"
df = pd.read_csv(data_path)

# Display the first few rows
print(df.head())

                                                text            timestamp  \
0                 id of j2gwp5d said this: The best.  2023-01-01 04:51:28   
1  id of j2i2mtw said this: Wah heart pain.\nid o...  2023-01-01 14:17:49   
2  id of j2h2alh said this: HNY DT where to go to...  2023-01-01 05:48:18   
3  id of j2hpea5 said this: Clubbing shenanigans ...  2023-01-01 11:08:43   
4  id of j2hewei said this: Oooffwas supposed to ...  2023-01-01 10:55:13   

            username                                               link  \
0       MisoMesoMilo  /r/singapore/comments/1004s1o/rsingapore_rando...   
1           crisseur  /r/singapore/comments/1004s1o/rsingapore_rando...   
2             dazark  /r/singapore/comments/1004s1o/rsingapore_rando...   
3  N1_Procrastinator  /r/singapore/comments/1004s1o/rsingapore_rando...   
4    EaeleButEeelier  /r/singapore/comments/1004s1o/rsingapore_rando...   

      link_id   parent_id       id subreddit_id  \
0  t3_1004s1o  t1_j2gje0f  j2gwp5d 

In [28]:
# Take a random sample of 100 rows for experimentation
subset_df = df.sample(n=10000, random_state=42)
subset_df = df.head(100)
subset_df.to_csv("/Users/sarahSTACS/Desktop/concatenated_texts_subset.csv", index=False)

In [29]:
import re


def clean_text(text):
    match = re.search(r"this:(.*)", text)
    if match:
        return match.group(1).strip()
    return ""


subset_df["clean_text"] = subset_df["text"].apply(clean_text)
print(subset_df[["text", "clean_text"]].head())
subset_df.to_csv(
    "/Users/sarahSTACS/DSA4264-Detoxify/kedro-data/data/03_primary/concatenated_texts_clean.csv",
    index=False,
)

                                                text  \
0                 id of j2gwp5d said this: The best.   
1  id of j2i2mtw said this: Wah heart pain.\nid o...   
2  id of j2h2alh said this: HNY DT where to go to...   
3  id of j2hpea5 said this: Clubbing shenanigans ...   
4  id of j2hewei said this: Oooffwas supposed to ...   

                                          clean_text  
0                                          The best.  
1                                    Wah heart pain.  
2  HNY DT where to go to repair or rewire a porta...  
3  Clubbing shenanigans some trips abroad and new...  
4  Oooffwas supposed to have a date today Think I...  


In [30]:
texts = subset_df["clean_text"].tolist()
from bertopic import BERTopic

# Initialize BERTopic model
topic_model = BERTopic()

# Fit and transform the subset of texts
topics, probs = topic_model.fit_transform(texts)

# Print the topics
print(topics)

[-1, 0, -1, 1, 0, 0, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 0, -1, 0, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, -1, 0, -1, 0, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, 0, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, 0, 0, 1, -1, 0, -1, 0, 1, -1, -1, -1, -1, 0, 0, -1, -1, 1, -1, 0, -1, -1, -1, 0]


In [31]:
topic_model.get_topic_info()
topic_model.get_topic(0)

[('year', np.float64(0.10479980336451349)),
 ('to', np.float64(0.1033162090776176)),
 ('you', np.float64(0.0982231788266198)),
 ('new', np.float64(0.09121540950912217)),
 ('the', np.float64(0.06345091978322862)),
 ('and', np.float64(0.06304140692510626)),
 ('so', np.float64(0.053150910313292506)),
 ('for', np.float64(0.050531723116544906)),
 ('thank', np.float64(0.0476831721653735)),
 ('but', np.float64(0.045607704754561086))]

In [33]:
## increase coherence, reduce stopwords
from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

# Fit and transform the subset of texts
topics, probs = topic_model.fit_transform(texts)

# Print the topics
print(topics)

[-1, 0, -1, 1, -1, 0, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 0, -1, 0, -1, 0, -1, -1, -1, 1, 0, 0, -1, 1, 0, 1, -1, -1, 0, -1, 0, 1, -1, -1, 0, 1, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 0, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, 0, 0, 1, 0, 0, 1, 0, -1, -1, 1, -1, 1, 0, -1, -1, -1, 1, -1, 0, 1, -1, -1, 0]


In [36]:
topic_model.get_topic_info()
topic_model.get_topic(0)

[('yourself', np.float32(0.3406423)),
 ('wishing', np.float32(0.31482232)),
 ('myself', np.float32(0.31322408)),
 ('feel', np.float32(0.29367647)),
 ('mute', np.float32(0.27056736)),
 ('thankful', np.float32(0.27039632)),
 ('happy', np.float32(0.25804138)),
 ('selfreflection', np.float32(0.2413726)),
 ('hope', np.float32(0.23538871)),
 ('being', np.float32(0.23393749))]